# Vers un développement plus simple

---

## Recompilation automatique du package

> La commande `npm run serve` utilisée au chapitre précédent permet de 'recompiler' la page web dès que les fichiers index.js ou autres sont modifiés. Seulement, ce n'est pas le cas pour le code web assembly de notre package écrit en Rust.

- La solution consiste à utiliser la commande `cargo watch` sur nos sources que l'on peut installer ainsi :

`cargo install cargo-watch`

- Bien qu'en général pour vos projets Rust cette commande s'utilisera sans aucun argument, ici pour le cas du développement pour le web, on lui donnera en suffixe la commande à exécuter pour recompiler en web assembly quand changent les sources :

`cargo watch -w src/ -s "wasm-pack build --target bundler"`

![WATCH](pictures/cargo-watch.png)

Utilise conjointement avec `npm run serve` sur le site, La modification du bundler aura pour effet de relancer une génération de la page web :

![SERVE](pictures/npm_run_serve.png)

### Régler les problèmes de mise en cache de webpack-dev

> Afin d'éviter certains problèmes de mise en cache des fichiers .wasm, on modifiera le fichier `index.js` en important directement le module first-step. Ce n'est pas une bonne pratique, mais je n'ai pas trouvé de solution plus satisfaisante avec les dernières versions de webpack-dev :

**Ainsi, le fichier index.js ... :**

```
import { greet } from 'first-step';
greet("WebAssembly with npm");
```

**... devient donc :**

```
import("./node_modules/first-step/first_step.js").then((mod) => {
    mod.greet("WebAssembly with npm");
});
```
*Si quelqu'un a une meilleure solution à me suggérer, je suis preneur.*

## Utilisation de console.log pour déboguer

### namespace

> Lors de l'appel a la fonction alert() de javascript, nous utilisons le namespace par défaut `window`, cependant, pour afficher quelque chose dans la console.log du navigateur web, il faudrait que nous puissions accéder à `console`. The `wasm-bindgen` Guide explique comment s'adresser à un certain namespace dans le chapitre 2.18.1.9 :

![NAMESPACE](pictures/guide_namespace.png)

L'appel à la fonction log de console est d'ailleurs décrit dans l'exemple :
```
#[wasm_bindgen]
extern "C" {
    #[wasm_bindgen(js_namespace = console)]
    fn log(s: &str);
}
```
*Ajouter ce bloc dans le fichier lib.rs.*

*Ainsi, l'utilisation dans le code de l'instruction `log("Hello WebAssembly");` affichera la chaine "Hello WebAssembly" dans la console du navigateur.*

### L'équivalent de println!

- À ce moment-là, il peut être intéressant de construire une macro `console_log!` afin de pouvoir bénéficier des mêmes fonctionnalités que pour `println!()`. Vous pouvez vous référer au chapitre 10 qui concerne les macros, la documentation de la macro `format_args!` est aussi utile :

```
macro_rules! console_log {
    ($($t:tt)*) => (log(&format_args!($($t)*).to_string()))
}
```

```
console_log!("Hello, {}!", name);
console_log!("Les {} sont {}", "carottes", "cuites");
console_log!("test");
```

## Exercices

### À l'aide de la documentation des macros chapitre 10, proposez un équivalent de la macro dbg! :

```
macro_rules! console_dbg {
    ($t:tt) => (console_log!("{:?}", $t))
} 
```

*Remarquez la petite différence dans son utilisation...*

```
#[derive(Debug)]
struct S {
    i: usize,
    j: usize,
}
let mut s = S { i: 3, j: 65 };
console_dbg!(s);
s.i += 1;
console_dbg!(s); 
```